<a href="https://colab.research.google.com/github/sarvenaz-vsl/Electrical-Vehicle-Adoption-Canada/blob/main/Electrical_Vehicle_Adoption_Canada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EV Adoption & Infrastructure in Canada (2017–2025)

**Goal.** Analyze the growth of zero-emission vehicles (ZEVs) and charging infrastructure across Canadian provinces and normalize by population for fair comparisons. Prepare a single, tidy CSV for Tableau.

**Data Sources**
- EV chargers: Natural Resources Canada (NRCan) – Alternative Fuelling Stations Locator  
  https://natural-resources.canada.ca/energy-efficiency/transportation-energy-efficiency/electric-charging-alternative-fuelling-stationslocator-map#/analyze?country=CA&tab=fuel&ev_levels=all&fuel=ELEC
- Population by province/sex (annual): Statistics Canada – Table 17-10-0005-01  
  https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=1710000501
- ZEV (quarterly/annual proxy): Statistics Canada – Table 20-10-0025-01  
  https://www150.statcan.gc.ca/t1/tbl1/en/cv.action?pid=2010002501

**What this notebook does.**

Cleans the three raw datasets, audits missingness, prepares annual EV/ZEV metrics, and exports a tidy CSV for Tableau.

## Setup
We import standard libraries and set file paths. Nothing here changes data; it only prepares the environment.


In [1]:
# Setup: imports, notebook options, and file paths
import os, re, csv, math, json
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.options.mode.copy_on_write = True  # cleaner pandas behavior

# File paths
RAW_ZEV = Path('/content/data/raw/raw_zev_quarterly.csv')
RAW_CHARGERS = Path('/content/data/raw/raw_chargers.csv')
RAW_POP = Path('/content/data/raw/raw_population_province.csv')

OUT_DIR = Path('/content/data/processed/')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Display: wider tables so previews are readable
pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 160)

### Sanity check — input files
Quick check that each input path exists before start.

In [2]:
for p in [RAW_ZEV, RAW_CHARGERS, RAW_POP]:
    print(f"{p}  | exists={p.exists()} " if p.exists() else f"{p}  | exists=False")

/content/data/raw/raw_zev_quarterly.csv  | exists=True 
/content/data/raw/raw_chargers.csv  | exists=True 
/content/data/raw/raw_population_province.csv  | exists=True 


## 1) Chargers — dataset overview (openings by province & year)

**What’s in here?** Annual EV charging **openings** for each province/territory.

**What we do in this section (non-destructive):**
- Confirm columns, row counts, and missingness


### Chargers — structure & missingness
We take a quick look at the chargers dataset (openings by province/year). This cell confirms shapes, columns, and missing values so we know what we’re dealing with.

In [3]:
# Load chargers data (no transformations here)
chargers = pd.read_csv(RAW_CHARGERS, low_memory=False)

# Normalize common missing tokens to actual NaN (keeps later math honest)
MISSING_TOKENS = ["", " ", ".", "..", "...", "NA", "N/A", "n/a", "NaN", "nan", "NULL", "null", "-", "--", "—"]
chargers = chargers.replace(MISSING_TOKENS, np.nan)

# Build a per-column missingness summary (%), then add aligned counts and dtypes
missing_summary = (
    chargers.isna()
    .mean()
    .mul(100)
    .reset_index()
    .rename(columns={"index": "Column", 0: "% Missing"})
    .sort_values("% Missing", ascending=False)
)

non_null = chargers.notna().sum()
dtypes   = chargers.dtypes.astype(str)

missing_summary["Non-Null Count"] = missing_summary["Column"].map(non_null)
missing_summary["Total Rows"]     = len(chargers)
missing_summary["Data Type"]      = missing_summary["Column"].map(dtypes)

# Clean preview of the top-most missing columns (display-only)
display(
    missing_summary.head(15)
    # .style.hide(axis="index").set_caption("Chargers — Top 15 columns by % missing")  # optional, cosmetic
)
print(f"Dataset shape: {chargers.shape}")

,Column,% Missing,Non-Null Count,Total Rows,Data Type
20,EV Other Info,100.0,0,36501,float64
14,BD Blends,100.0,0,36501,float64
15,NG Fill Type Code,100.0,0,36501,float64
10,Expected Date,100.0,0,36501,float64
7,Plus4,100.0,0,36501,float64
16,NG PSI,100.0,0,36501,float64
56,LPG Nozzle Types,100.0,0,36501,float64
58,Hydrogen Standards,100.0,0,36501,float64
57,Hydrogen Pressures,100.0,0,36501,float64
59,CNG Fill Type Code,100.0,0,36501,float64


Dataset shape: (36501, 85)


In [4]:
# Drop columns with >95% missing values (keep only dense/useful columns)
threshold = 95  # percent
keep_cols = missing_summary.loc[missing_summary["% Missing"] <= threshold, "Column"]

# Keep a reduced copy for further work; original 'chargers' remains intact
chargers_reduced = chargers[keep_cols].copy()
print(f"\nKept {chargers_reduced.shape[1]} columns (dropped {chargers.shape[1] - chargers_reduced.shape[1]} sparse ones)")

# Re-check remaining missingness
missing_after = (
    chargers_reduced.isna()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=False)
)

# display(missing_after.to_frame(name="% Missing").style.hide(axis="index").set_caption("Missingness after drop"))
display(missing_after.head(15))


Kept 45 columns (dropped 40 sparse ones)


,0
EV J3400 Power Output (kW),92.63
EV CHAdeMO Power Output (kW),92.33
Funding Sources,90.29
Intersection Directions,89.33
Cards Accepted,89.07
EV CCS Power Output (kW),87.30
Maximum Vehicle Class,84.15
EV Pricing (French),84.07
EV Pricing,84.04
Facility Type,79.89


In [5]:
# all column names renamed in chargers_reduced
print(f"Columns in chargers_reduced ({len(chargers_reduced.columns)} total):\n")

for i, c in enumerate(chargers_reduced.columns, start=1):
    print(f"{i:>2}. {c}")

Columns in chargers_reduced (45 total):

 1. EV J3400 Power Output (kW)
 2. EV CHAdeMO Power Output (kW)
 3. Funding Sources
 4. Intersection Directions
 5. Cards Accepted
 6. EV CCS Power Output (kW)
 7. Maximum Vehicle Class
 8. EV Pricing (French)
 9. EV Pricing
10. Facility Type
11. EV DC Fast Count
12. Owner Type Code
13. EV J1772 Power Output (kW)
14. Access Days Time (French)
15. Access Days Time
16. EV Level2 EVSE Num
17. Restricted Access
18. EV Network Web
19. Station Phone
20. Date Last Confirmed
21. Open Date
22. Geocode Status
23. EV Workplace Charging
24. Status Code
25. Groups With Access Code
26. Fuel Type Code
27. Station Name
28. Street Address
29. City
30. State
31. ZIP
32. EV Connector Types
33. Access Code
34. EV Network
35. ID
36. Longitude
37. Latitude
38. Updated At
39. Country
40. Groups With Access Code (French)
41. EV J1772 Connector Count
42. EV CHAdeMO Connector Count
43. EV CCS Connector Count
44. EV J3400 Connector Count
45. EV J3271 Connector Count


We subset to just the columns we need for downstream processing (ID, location, fuel type, counts, and open date). Then we clean obvious gaps: drop rows with missing open date or location, and normalize EV count fields.

In [6]:
# Choose the specific fields used downstream (ID, location, counts, dates)
keep_cols = [
    "ID", "Station Name", "Fuel Type Code",
    "City", "State", "Latitude", "Longitude",
    "EV Level2 EVSE Num", "EV DC Fast Count",
    "Access Code", "Status Code", "Open Date"
]

# Subset without altering original 'chargers_reduced'
chargers_final = chargers_reduced[keep_cols].copy()
print(f"Retained {len(keep_cols)} columns, {chargers_final.shape[0]:,} rows")

# Require an open date for timeline analysis
chargers_final = chargers_final.dropna(subset=["Open Date"])

# Normalize EV count fields (fill blanks with 0, cast to int)
for col in ["EV Level2 EVSE Num", "EV DC Fast Count"]:
    if col in chargers_final.columns:
        chargers_final[col] = chargers_final[col].fillna(0).astype(int)

# Require basic location fields so we can place stations on the map
chargers_final = chargers_final.dropna(subset=["State", "City", "Latitude", "Longitude"], how="any")

# Parse Open Date to datetime (NaT means an unparseable value)
chargers_final["Open Date"] = pd.to_datetime(chargers_final["Open Date"], errors="coerce")

# Show remaining missingness (percentage) for awareness only
missing_final = (
    chargers_final.isna()
                 .mean()
                 .mul(100)
                 .round(2)
                 .sort_values(ascending=False)
)
display(missing_final)

# Preview sample rows (display-only)
chargers_final.head(10)

Retained 12 columns, 36,501 rows


,0
ID,0.0
Station Name,0.0
Fuel Type Code,0.0
City,0.0
State,0.0
Latitude,0.0
Longitude,0.0
EV Level2 EVSE Num,0.0
EV DC Fast Count,0.0
Access Code,0.0


,ID,Station Name,Fuel Type Code,City,State,Latitude,Longitude,EV Level2 EVSE Num,EV DC Fast Count,Access Code,Status Code,Open Date
0,82833,Ramada,ELEC,Brooks,AB,50.585242,-111.898615,1,0,public,E,2012-02-01
1,82834,Davis Chevrolet,ELEC,Airdrie,AB,51.288119,-113.998284,1,0,public,E,2015-01-15
2,82837,Gasonic Instruments,ELEC,Calgary,AB,51.092856,-114.043029,1,0,public,E,2015-04-15
3,82838,International Motor Cars,ELEC,Calgary,AB,50.990495,-114.042414,1,0,public,E,2017-03-15
4,82839,Residence Inn,ELEC,Calgary,AB,50.880283,-113.955873,4,0,public,E,2017-02-01
5,82839,Residence Inn,ELEC,Calgary,AB,50.880283,-113.955873,4,0,public,E,2017-02-01
6,82839,Residence Inn,ELEC,Calgary,AB,50.880283,-113.955873,4,0,public,E,2017-02-01
7,82839,Residence Inn,ELEC,Calgary,AB,50.880283,-113.955873,4,0,public,E,2017-02-01
8,82840,Platinum Mitsubishi,ELEC,Calgary,AB,51.076263,-114.000260,2,0,public,E,2016-05-16
9,82840,Platinum Mitsubishi,ELEC,Calgary,AB,51.076263,-114.000260,2,0,public,E,2016-05-16


Quick look at the distinct values in the “State” field


In [7]:
# Quick glance at distinct region codes (display-only)
chargers_final['State'].unique()

array(['AB', 'QC', 'ON', 'MB', 'NL', 'PE', 'NB', 'SK', 'NS', 'BC', 'YT',
       'NT'], dtype=object)

We keep **electric** stations only and require **active** status (Status Code = 'E') so the infrastructure counts reflect currently available charging.

In [8]:
# Keep only electric stations
chargers_final = chargers_final[chargers_final["Fuel Type Code"] == "ELEC"]

# Keep only active stations (E = Existing/Active)
chargers_final = chargers_final[chargers_final["Status Code"] == "E"]

# (display-only) Show remaining row count for awareness
print(f"Rows after ELEC + active filter: {len(chargers_final):,}")

Rows after ELEC + active filter: 36,430


Duplicates and ID consistency

We check how many rows share the same station `ID`, whether any IDs appear multiple times, and whether a single `ID` ever shows conflicting location/name info. This is read-only; we won’t modify data yet.

In [9]:
# How many unique station IDs vs total rows?
n_rows = len(chargers_final)
n_ids  = chargers_final["ID"].nunique()
print(f"Rows: {n_rows:,} | Unique IDs: {n_ids:,} | Duplicate IDs: {n_rows - n_ids:,}")

# Distribution: how many times each ID appears (top 10 for a quick sense)
dup_dist = (chargers_final.groupby("ID").size()
            .reset_index(name="rows_per_id")
            .sort_values("rows_per_id", ascending=False))

# (display-only) Compact view
display(dup_dist.head(10).style.hide(axis="index").set_caption("Rows per ID — top 10"))

Rows: 36,430 | Unique IDs: 13,892 | Duplicate IDs: 22,538


ID,rows_per_id
377588,86
305734,60
116556,60
360066,54
345872,50
312524,40
360004,39
305196,34
116559,32
321180,32


ID/location consistency check

If a single `ID` maps to multiple names, cities, or coordinates, that indicates a source inconsistency. We just count those cases here.

In [10]:
# Does the same ID ever have different name/location fields?
loc_check = (chargers_final
    .groupby("ID")
    .agg(n_name=("Station Name","nunique"),
         n_city=("City","nunique"),
         n_state=("State","nunique"),
         n_lat=("Latitude","nunique"),
         n_lon=("Longitude","nunique"))
    .reset_index())

inconsistent = loc_check.query("n_name>1 or n_city>1 or n_state>1 or n_lat>1 or n_lon>1")
print(f"Inconsistent IDs (same ID but different name/location): {len(inconsistent):,}")

# (display-only) Show just a few examples if any exist
if len(inconsistent):
    display(inconsistent.head(10).style.hide(axis="index").set_caption("Examples: inconsistent IDs"))

Inconsistent IDs (same ID but different name/location): 0


Keep the most relevant record per ID

We keep one row per `ID`, preferring the **latest** `Open Date` and the **larger** EVSE counts when there are ties. This produces a single, up-to-date record per station.

In [11]:
# Keep the most relevant record per ID:
# 1) latest Open Date, 2) larger Level2 count, 3) larger DC Fast count
chargers_final = (
    chargers_final
    .sort_values(["ID", "Open Date", "EV Level2 EVSE Num", "EV DC Fast Count"],
                 ascending=[True, False, False, False])
    .drop_duplicates(subset="ID", keep="first")
    .reset_index(drop=True)
)

print(f"After removing duplicate IDs: {len(chargers_final):,} rows remaining")

# (diagnostic only) verify uniqueness now
dups_left = chargers_final["ID"].duplicated().sum()
if dups_left:
    print(f"Still duplicated IDs after dedup: {dups_left}")

chargers_final.head()


After removing duplicate IDs: 13,892 rows remaining


,ID,Station Name,Fuel Type Code,City,State,Latitude,Longitude,EV Level2 EVSE Num,EV DC Fast Count,Access Code,Status Code,Open Date
0,82833,Ramada,ELEC,Brooks,AB,50.585242,-111.898615,1,0,public,E,2012-02-01
1,82834,Davis Chevrolet,ELEC,Airdrie,AB,51.288119,-113.998284,1,0,public,E,2015-01-15
2,82837,Gasonic Instruments,ELEC,Calgary,AB,51.092856,-114.043029,1,0,public,E,2015-04-15
3,82838,International Motor Cars,ELEC,Calgary,AB,50.990495,-114.042414,1,0,public,E,2017-03-15
4,82839,Residence Inn,ELEC,Calgary,AB,50.880283,-113.955873,4,0,public,E,2017-02-01


Extract year/month from open date

We’ll use `Open Year` and `Open Month` for yearly/seasonal analyses and to align with other annual series later.

In [12]:
# Ensure datetime dtype (safe even if already parsed earlier)
chargers_final["Open Date"] = pd.to_datetime(chargers_final["Open Date"], errors="coerce")

# Extract year/month components
chargers_final["Open Year"] = chargers_final["Open Date"].dt.year
chargers_final["Open Month"] = chargers_final["Open Date"].dt.month

# Quick validation
print(chargers_final[["Open Date", "Open Year", "Open Month"]].head())
print("\nYear range:", chargers_final["Open Year"].min(), "-", chargers_final["Open Year"].max())
print("Month values:", sorted(chargers_final["Open Month"].dropna().unique()))

# (diagnostic only) If any NaT slipped through, call it out
nat_post = chargers_final["Open Date"].isna().sum()
if nat_post:
    print(f"{nat_post} rows have NaT in 'Open Date' after parsing. Consider dropping them.")

   Open Date  Open Year  Open Month
0 2012-02-01       2012           2
1 2015-01-15       2015           1
2 2015-04-15       2015           4
3 2017-03-15       2017           3
4 2017-02-01       2017           2

Year range: 2009 - 2025
Month values: [np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(9), np.int32(10), np.int32(11), np.int32(12)]


In [13]:
# === Chargers yearly (2017–2024) — seed 2016 baseline and build L2/DC fast cumulative ===
PROV_MAP = {
    "AB":"Alberta","BC":"British Columbia","MB":"Manitoba","NB":"New Brunswick",
    "NL":"Newfoundland and Labrador","NS":"Nova Scotia","NT":"Northwest Territories",
    "NU":"Nunavut","ON":"Ontario","PE":"Prince Edward Island","QC":"Quebec",
    "SK":"Saskatchewan","YT":"Yukon"
}
YEARS = list(range(2017, 2025))  # 2017..2024

cf = chargers_final.copy()
cf["geo"]    = cf["State"].map(PROV_MAP)
cf["level2"] = pd.to_numeric(cf["EV Level2 EVSE Num"], errors="coerce").fillna(0).astype(int)
cf["dcfast"] = pd.to_numeric(cf["EV DC Fast Count"],  errors="coerce").fillna(0).astype(int)

# Openings for ALL years (to build baseline)
all_open = (cf.groupby(["geo","Open Year"], as_index=False)
              .agg(stations_opened=("ID","nunique"),
                   level2_opened=("level2","sum"),
                   dcfast_opened=("dcfast","sum"))
              .rename(columns={"Open Year":"year"})
              .sort_values(["geo","year"]))

# Baseline ≤2016 as a seed row @ 2016 (OPENINGS = baseline)
base = (all_open.query("year <= 2016")
          .groupby("geo", as_index=False)
          .agg(stations_opened=("stations_opened","sum"),
               level2_opened   =("level2_opened","sum"),
               dcfast_opened   =("dcfast_opened","sum"))
          .assign(year=2016))

# Stack seed + window (2017–2024) then cumulative per geo
win = all_open.query("2017 <= year <= 2024")[["geo","year","stations_opened","level2_opened","dcfast_opened"]]
stacked = (pd.concat([base[["geo","year","stations_opened","level2_opened","dcfast_opened"]], win], ignore_index=True)
             .sort_values(["geo","year"]))

stacked["chargers_stations"] = stacked.groupby("geo")["stations_opened"].cumsum().astype(int)
stacked["level2_ports"]      = stacked.groupby("geo")["level2_opened"].cumsum().astype(int)
stacked["dcfast_ports"]      = stacked.groupby("geo")["dcfast_opened"].cumsum().astype(int)
stacked["chargers_ports"]    = stacked["level2_ports"] + stacked["dcfast_ports"]

# Keep 2017–2024
chargers_clean = stacked.query("2017 <= year <= 2024").copy()

# Full geo×year grid, fill openings=0, carry cumulative
grid = pd.MultiIndex.from_product([chargers_clean["geo"].dropna().unique(), YEARS], names=["geo","year"]).to_frame(index=False)
chargers_clean = (grid.merge(chargers_clean, on=["geo","year"], how="left")
                      .sort_values(["geo","year"]))

for c in ["stations_opened","level2_opened","dcfast_opened"]:
    chargers_clean[c] = chargers_clean[c].fillna(0).astype(int)
for c in ["chargers_stations","level2_ports","dcfast_ports","chargers_ports"]:
    chargers_clean[c] = chargers_clean.groupby("geo")[c].ffill().fillna(0).astype(int)

# fast_share (safe)
num = pd.to_numeric(chargers_clean["dcfast_ports"],  errors="coerce")
den = pd.to_numeric(chargers_clean["chargers_ports"], errors="coerce")
chargers_clean["fast_share"] = num.div(den).where(den > 0)

display(chargers_clean.head(12))

,geo,year,stations_opened,level2_opened,dcfast_opened,chargers_stations,level2_ports,dcfast_ports,chargers_ports,fast_share
0,Alberta,2017,14,29,0,64,123,16,139,0.115108
1,Alberta,2018,10,31,0,74,154,16,170,0.094118
2,Alberta,2019,30,94,23,104,248,39,287,0.135889
3,Alberta,2020,22,29,30,126,277,69,346,0.199422
4,Alberta,2021,51,108,37,177,385,106,491,0.215886
5,Alberta,2022,153,280,45,330,665,151,816,0.185049
6,Alberta,2023,210,434,91,540,1099,242,1341,0.180462
7,Alberta,2024,166,323,93,706,1422,335,1757,0.190666
8,British Columbia,2017,42,87,19,193,398,49,447,0.109620
9,British Columbia,2018,68,126,47,261,524,96,620,0.154839


Save cleaned chargers

We save the cleaned station-level table. No Tableau-specific renaming yet; we’ll do column renames only at the final export step.

In [14]:
# Save the fully cleaned chargers dataset (station-level)
chargers_clean.to_csv(OUT_DIR / "chargers_processed.csv", index=False)
print("Saved to: {OUT_DIR / 'chargers_processed.csv'}")
display(chargers_clean.head(12))

Saved to: {OUT_DIR / 'chargers_processed.csv'}


,geo,year,stations_opened,level2_opened,dcfast_opened,chargers_stations,level2_ports,dcfast_ports,chargers_ports,fast_share
0,Alberta,2017,14,29,0,64,123,16,139,0.115108
1,Alberta,2018,10,31,0,74,154,16,170,0.094118
2,Alberta,2019,30,94,23,104,248,39,287,0.135889
3,Alberta,2020,22,29,30,126,277,69,346,0.199422
4,Alberta,2021,51,108,37,177,385,106,491,0.215886
5,Alberta,2022,153,280,45,330,665,151,816,0.185049
6,Alberta,2023,210,434,91,540,1099,242,1341,0.180462
7,Alberta,2024,166,323,93,706,1422,335,1757,0.190666
8,British Columbia,2017,42,87,19,193,398,49,447,0.109620
9,British Columbia,2018,68,126,47,261,524,96,620,0.154839


In [15]:
# Quick QA (tiny, safe)
t = chargers_clean.sort_values(["geo","year"]).copy()
assert t['year'].between(2017, 2024).all()
assert t.groupby(['geo','year']).size().max() == 1  # no duplicate keys
diff = t.groupby('geo')[['chargers_stations','chargers_ports']].diff().fillna(0)
assert (diff >= 0).all().all()  # cumulative never decreases
print("Chargers processed file looks good ")

Chargers processed file looks good 


# ZEV Dataset

### ZEV dataset — load & quick shape
Load the raw StatCan ZEV file.


In [16]:
zev_raw = pd.read_csv(RAW_ZEV)
print("Raw shape:", zev_raw.shape)
zev_raw.head(3)

Raw shape: (19040, 17)


,REF_DATE,GEO,DGUID,Fuel type,Vehicle type,Statistics,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2017-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,425031.0,NaN,NaN,NaN,0
1,2017-04,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,610662.0,NaN,NaN,NaN,0
2,2017-07,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,551572.0,NaN,NaN,NaN,0


Normalize columns and quick inspect of key dimensions

In [17]:
zev = zev_raw.copy()
zev.columns = (
    zev.columns.str.strip().str.replace(r"\s+", "_", regex=True).str.lower()
)

# Show available columns once
print(sorted(zev.columns))

# Common StatCan columns (robust to slight name variations)
col_ref   = "ref_date"
col_geo   = "geo"
col_vtype = "vehicle_type"
col_ftype = "fuel_type"
col_stat  = "statistics"
col_value = "value"

missing_cols = [c for c in [col_ref, col_geo, col_vtype, col_ftype, col_stat, col_value] if c not in zev.columns]
if missing_cols:
    raise KeyError(f"Expected columns not found: {missing_cols}")

print("\nUnique samples:")
print("• geo:", zev[col_geo].dropna().unique()[:8])
print("• vehicle_type:", zev[col_vtype].dropna().unique()[:8])
print("• fuel_type:", zev[col_ftype].dropna().unique()[:8])
print("• statistics:", zev[col_stat].dropna().unique()[:8])

print("\nPreview:")
zev.head(5)


['coordinate', 'decimals', 'dguid', 'fuel_type', 'geo', 'ref_date', 'scalar_factor', 'scalar_id', 'statistics', 'status', 'symbol', 'terminated', 'uom', 'uom_id', 'value', 'vector', 'vehicle_type']

Unique samples:
• geo: ['Canada' 'Newfoundland and Labrador' 'Prince Edward Island' 'Nova Scotia'
 'New Brunswick' 'Quebec' 'Ontario' 'Manitoba']
• vehicle_type: ['Total, vehicle type' 'Passenger cars' 'Pickup trucks'
 'Multi-purpose vehicles' 'Vans']
• fuel_type: ['All fuel types' 'Gasoline' 'Diesel' 'All zero-emission vehicles'
 'Battery electric' 'Plug-in hybrid electric' 'Hybrid electric'
 'Other fuel types']
• statistics: ['Number of vehicles']

Preview:


,ref_date,geo,dguid,fuel_type,vehicle_type,statistics,uom,uom_id,scalar_factor,scalar_id,vector,coordinate,value,status,symbol,terminated,decimals
0,2017-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,425031.0,NaN,NaN,NaN,0
1,2017-04,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,610662.0,NaN,NaN,NaN,0
2,2017-07,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,551572.0,NaN,NaN,NaN,0
3,2017-10,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,450479.0,NaN,NaN,NaN,0
4,2018-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,406109.0,NaN,NaN,NaN,0


Basic filters (province list, vehicle type present, 'Number of vehicles')

Keep rows that match StatCan province names, have a vehicle type, and use the “Number of vehicles” statistic.

In [18]:
# Define valid provinces (matching StatCan naming)
PROVINCES = {
    "Canada",
    "Newfoundland and Labrador", "Prince Edward Island", "Nova Scotia", "New Brunswick",
    "Quebec", "Ontario", "Manitoba", "Saskatchewan", "Alberta", "British Columbia",
    "Yukon", "Northwest Territories", "Nunavut"
}

# Apply filters
mask_geo = zev["geo"].isin(PROVINCES)
mask_vtype = zev["vehicle_type"].notna()
mask_stat = zev["statistics"].eq("Number of vehicles")

zev_filt = zev.loc[mask_geo & mask_vtype & mask_stat].copy()

print("After basic filters:", zev_filt.shape)

# Preview
zev_filt.head()

After basic filters: (19040, 17)


,ref_date,geo,dguid,fuel_type,vehicle_type,statistics,uom,uom_id,scalar_factor,scalar_id,vector,coordinate,value,status,symbol,terminated,decimals
0,2017-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,425031.0,NaN,NaN,NaN,0
1,2017-04,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,610662.0,NaN,NaN,NaN,0
2,2017-07,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,551572.0,NaN,NaN,NaN,0
3,2017-10,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,450479.0,NaN,NaN,NaN,0
4,2018-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671330686,1.4.1.1,406109.0,NaN,NaN,NaN,0


Quick structure & missingness

Show counts and small samples of key categoricals, plus a concise missingness summary.

In [19]:
# Quick overview of the key categorical fields
print("Unique values per key column:\n")
print("geo:", zev_filt["geo"].nunique(), "-", sorted(zev_filt["geo"].unique().tolist()))
print("vehicle_type:", zev_filt["vehicle_type"].unique().tolist())
print("fuel_type:", zev_filt["fuel_type"].unique().tolist())

# Missing value summary
print("\nMissing values per column (%):")
missing_summary = (
    zev_filt.isna().mean().round(4) * 100
).sort_values(ascending=False)
print(missing_summary)

# Show any rows with missing 'value'
print("\nRows with missing 'value':")
zev_filt[zev_filt["value"].isna()].head()

Unique values per key column:

geo: 14 - ['Alberta', 'British Columbia', 'Canada', 'Manitoba', 'New Brunswick', 'Newfoundland and Labrador', 'Northwest Territories', 'Nova Scotia', 'Nunavut', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan', 'Yukon']
vehicle_type: ['Total, vehicle type', 'Passenger cars', 'Pickup trucks', 'Multi-purpose vehicles', 'Vans']
fuel_type: ['All fuel types', 'Gasoline', 'Diesel', 'All zero-emission vehicles', 'Battery electric', 'Plug-in hybrid electric', 'Hybrid electric', 'Other fuel types']

Missing values per column (%):
symbol           100.00
terminated       100.00
status            81.07
value             18.93
ref_date           0.00
geo                0.00
dguid              0.00
fuel_type          0.00
vehicle_type       0.00
scalar_factor      0.00
uom_id             0.00
uom                0.00
statistics         0.00
coordinate         0.00
vector             0.00
scalar_id          0.00
decimals           0.00
dtype: float64

Rows wi

,ref_date,geo,dguid,fuel_type,vehicle_type,statistics,uom,uom_id,scalar_factor,scalar_id,vector,coordinate,value,status,symbol,terminated,decimals
1360,2017-01,Newfoundland and Labrador,2021A000210,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671371793,2.4.1.1,NaN,..,NaN,NaN,0
1361,2017-04,Newfoundland and Labrador,2021A000210,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671371793,2.4.1.1,NaN,..,NaN,NaN,0
1362,2017-07,Newfoundland and Labrador,2021A000210,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671371793,2.4.1.1,NaN,..,NaN,NaN,0
1363,2017-10,Newfoundland and Labrador,2021A000210,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671371793,2.4.1.1,NaN,..,NaN,NaN,0
1364,2018-01,Newfoundland and Labrador,2021A000210,All fuel types,"Total, vehicle type",Number of vehicles,Units,300,units,0,v1671371793,2.4.1.1,NaN,..,NaN,NaN,0


Clean & simplify (drop non-essential metadata, normalize text, coerce value)

Reduce to analysis fields, standardize text, standardize missing tokens, and convert the metric to numeric.

In [20]:
# Columns to drop — not needed for analysis
drop_cols = [
    "symbol", "terminated", "status", "scalar_factor",
    "scalar_id", "uom_id", "decimals", "vector", "coordinate", "statistics"
]

zev_clean = zev_filt.drop(columns=drop_cols, errors="ignore").copy()

# Normalize text fields
text_cols = zev_clean.select_dtypes(include="object").columns
zev_clean[text_cols] = zev_clean[text_cols].apply(lambda c: c.str.strip())

# Replace '...' or '..' with NaN
zev_clean = zev_clean.replace(["..", "..."], np.nan)

# Convert value column to numeric
zev_clean["value"] = pd.to_numeric(zev_clean["value"], errors="coerce")

# Display structure
print("After cleaning:", zev_clean.shape)
print("Remaining columns:", zev_clean.columns.tolist())

# Missing value summary (after cleaning)
print("\nMissing values (%):")
print((zev_clean.isna().mean().round(4) * 100).sort_values(ascending=False))

# Check unique vehicle types
print("\nUnique vehicle types:", zev_clean["vehicle_type"].unique().tolist())

zev_clean.head()

After cleaning: (19040, 7)
Remaining columns: ['ref_date', 'geo', 'dguid', 'fuel_type', 'vehicle_type', 'uom', 'value']

Missing values (%):
value           18.93
geo              0.00
ref_date         0.00
dguid            0.00
fuel_type        0.00
vehicle_type     0.00
uom              0.00
dtype: float64

Unique vehicle types: ['Total, vehicle type', 'Passenger cars', 'Pickup trucks', 'Multi-purpose vehicles', 'Vans']


,ref_date,geo,dguid,fuel_type,vehicle_type,uom,value
0,2017-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Units,425031.0
1,2017-04,Canada,2021A000011124,All fuel types,"Total, vehicle type",Units,610662.0
2,2017-07,Canada,2021A000011124,All fuel types,"Total, vehicle type",Units,551572.0
3,2017-10,Canada,2021A000011124,All fuel types,"Total, vehicle type",Units,450479.0
4,2018-01,Canada,2021A000011124,All fuel types,"Total, vehicle type",Units,406109.0


Time fields and coverage

Parse `ref_date` to year/quarter fields and produce compact coverage tables by province and time.


In [21]:
# parse 'YYYY-MM' into datetime
ref_dt = pd.to_datetime(zev_clean["ref_date"], format="%Y-%m", errors="coerce")

# year / month / month name
zev_clean["Year"]       = ref_dt.dt.year.astype("Int64")
zev_clean["Month_Num"]  = ref_dt.dt.month.astype("Int64")
zev_clean["Month_Name"] = ref_dt.dt.strftime("%b")

# quarter num/label
zev_clean["Quarter_Num"] = ((ref_dt.dt.month - 1) // 3 + 1).astype("Int64")
zev_clean["Quarter"]     = "Q" + zev_clean["Quarter_Num"].astype(str) + " " + zev_clean["Year"].astype(str)

# quarter end date as YYYY-MM-DD (string)
zev_clean["Quarter_End"] = ref_dt.dt.to_period("Q").dt.end_time.dt.normalize().dt.strftime("%Y-%m-%d")

print(f"Time helpers added. Shape: {zev_clean.shape}")
print("Years:", zev_clean["Year"].min(), "-", zev_clean["Year"].max())
print("Vehicle types:", sorted(zev_clean["vehicle_type"].dropna().unique().tolist()))
print("Fuel types:",    sorted(zev_clean["fuel_type"].dropna().unique().tolist()))

# coverage

# by Province × Year
coverage_year = (zev_clean.groupby(["geo", "Year"], dropna=False)
                 .size().rename("rows").reset_index().sort_values(["geo","Year"]))
display(coverage_year.head().style.hide(axis="index").set_caption("Coverage: geo × Year"))

# by Province × Year × Quarter
coverage_qtr = (zev_clean.groupby(["geo", "Year", "Quarter_Num", "Quarter"], dropna=False)
                .size().rename("rows").reset_index().sort_values(["geo","Year","Quarter_Num"]))
display(coverage_qtr.head().style.hide(axis="index").set_caption("Coverage: geo × Year × Quarter"))

# by Province × Year × Month
coverage_month = (zev_clean.groupby(["geo", "Year", "Month_Num", "Month_Name"], dropna=False)
                  .size().rename("rows").reset_index().sort_values(["geo","Year","Month_Num"]))
display(coverage_month.head().style.hide(axis="index").set_caption("Coverage: geo × Year × Month"))

# compact sample of full data (selected columns only; deterministic ordering)
cols_show = ["ref_date", "Year", "Month_Num", "Month_Name", "Quarter",
             "geo", "fuel_type", "vehicle_type", "value"]
_display_sample = (zev_clean.loc[:, cols_show]
                   .sort_values(["geo","Year","Month_Num"])
                   .head())
display(_display_sample.style.hide(axis="index").set_caption("ZEV sample"))

Time helpers added. Shape: (19040, 13)
Years: 2017 - 2025
Vehicle types: ['Multi-purpose vehicles', 'Passenger cars', 'Pickup trucks', 'Total, vehicle type', 'Vans']
Fuel types: ['All fuel types', 'All zero-emission vehicles', 'Battery electric', 'Diesel', 'Gasoline', 'Hybrid electric', 'Other fuel types', 'Plug-in hybrid electric']


geo,Year,rows
Alberta,2017,160
Alberta,2018,160
Alberta,2019,160
Alberta,2020,160
Alberta,2021,160


geo,Year,Quarter_Num,Quarter,rows
Alberta,2017,1,Q1 2017,40
Alberta,2017,2,Q2 2017,40
Alberta,2017,3,Q3 2017,40
Alberta,2017,4,Q4 2017,40
Alberta,2018,1,Q1 2018,40


geo,Year,Month_Num,Month_Name,rows
Alberta,2017,1,Jan,40
Alberta,2017,4,Apr,40
Alberta,2017,7,Jul,40
Alberta,2017,10,Oct,40
Alberta,2018,1,Jan,40


ref_date,Year,Month_Num,Month_Name,Quarter,geo,fuel_type,vehicle_type,value
2017-01,2017,1,Jan,Q1 2017,Alberta,All fuel types,"Total, vehicle type",nan
2017-01,2017,1,Jan,Q1 2017,Alberta,All fuel types,Passenger cars,nan
2017-01,2017,1,Jan,Q1 2017,Alberta,All fuel types,Pickup trucks,nan
2017-01,2017,1,Jan,Q1 2017,Alberta,All fuel types,Multi-purpose vehicles,nan
2017-01,2017,1,Jan,Q1 2017,Alberta,All fuel types,Vans,nan


Metric missingness (non-destructive)

We annotate where `value` is missing without dropping rows, then summarize missingness overall, by province, and by year.

In [22]:
# helper flags (do not drop rows)
zev_clean["value_missing"]  = zev_clean["value"].isna()
zev_clean["value_filled_0"] = zev_clean["value"].fillna(0)

# overall missing stats
total_rows   = len(zev_clean)
missing_rows = int(zev_clean["value_missing"].sum())
missing_pct  = round(missing_rows / total_rows * 100, 2)

print("Missing `value` — overall")
print(f"  Total rows: {total_rows}")
print(f"  Missing rows: {missing_rows} ({missing_pct}%)")

# missing by province (percent)
missing_by_geo = (
    zev_clean.groupby("geo", dropna=False)["value_missing"]
             .mean()
             .mul(100).round(2)
             .rename("% missing")
             .reset_index()
             .sort_values("% missing", ascending=False)
)
# compact display
display(missing_by_geo.style.hide(axis="index").set_caption("Missing `value` by province (%)"))

# missing by year (percent)
missing_by_year = (
    zev_clean.groupby("Year", dropna=False)["value_missing"]
             .mean()
             .mul(100).round(2)
             .rename("% missing")
             .reset_index()
             .sort_values("Year")
)
# compact display
display(missing_by_year.style.hide(axis="index").set_caption("Missing `value` by year (%)"))

# preview a few rows where `value` is missing (display-only)
cols_show = ["ref_date", "Year", "Quarter", "geo", "fuel_type", "vehicle_type", "value"]
missing_rows_preview = (
    zev_clean.loc[zev_clean["value_missing"], cols_show]
             .sort_values(["geo", "Year", "Quarter"])
             .head()
)
display(missing_rows_preview.style.hide(axis="index").set_caption("Rows with missing `value`"))

Missing `value` — overall
  Total rows: 19040
  Missing rows: 3605 (18.93%)


geo,% missing
Alberta,100.000000
Newfoundland and Labrador,100.000000
Nunavut,65.070000
British Columbia,0.000000
Manitoba,0.000000
Canada,0.000000
New Brunswick,0.000000
Northwest Territories,0.000000
Nova Scotia,0.000000
Ontario,0.000000


Year,% missing
2017,21.430000
2018,21.430000
2019,21.430000
2020,21.430000
2021,21.430000
2022,16.290000
2023,14.290000
2024,14.290000
2025,17.860000


ref_date,Year,Quarter,geo,fuel_type,vehicle_type,value
2017-01,2017,Q1 2017,Alberta,All fuel types,"Total, vehicle type",nan
2017-01,2017,Q1 2017,Alberta,All fuel types,Passenger cars,nan
2017-01,2017,Q1 2017,Alberta,All fuel types,Pickup trucks,nan
2017-01,2017,Q1 2017,Alberta,All fuel types,Multi-purpose vehicles,nan
2017-01,2017,Q1 2017,Alberta,All fuel types,Vans,nan


Save a processed table (single master file)

We keep a clean set of columns from the filtered/cleaned data. No dropping here.

In [23]:
# desired column order (keep only those that exist)
cols_order = [
    "ref_date",
    "Year", "Month_Num", "Month_Name",
    "Quarter_Num", "Quarter", "Quarter_End",
    "geo", "fuel_type", "vehicle_type",
    "uom", "dguid",
    "value"
]
# keep only columns that exist (safety)
cols_keep = [c for c in cols_order if c in zev_clean.columns]
zev_processed = zev_clean[cols_keep].copy()

print("Columns:", zev_processed.columns.tolist())
print("Years:", zev_processed['Year'].min(), "-", zev_processed['Year'].max())

# compact display
display(
    zev_processed.head()
      .style.hide(axis="index")
      .set_caption("ZEV processed")
)

Columns: ['ref_date', 'Year', 'Month_Num', 'Month_Name', 'Quarter_Num', 'Quarter', 'Quarter_End', 'geo', 'fuel_type', 'vehicle_type', 'uom', 'dguid', 'value']
Years: 2017 - 2025


ref_date,Year,Month_Num,Month_Name,Quarter_Num,Quarter,Quarter_End,geo,fuel_type,vehicle_type,uom,dguid,value
2017-01,2017,1,Jan,1,Q1 2017,2017-03-31,Canada,All fuel types,"Total, vehicle type",Units,2021A000011124,425031.000000
2017-04,2017,4,Apr,2,Q2 2017,2017-06-30,Canada,All fuel types,"Total, vehicle type",Units,2021A000011124,610662.000000
2017-07,2017,7,Jul,3,Q3 2017,2017-09-30,Canada,All fuel types,"Total, vehicle type",Units,2021A000011124,551572.000000
2017-10,2017,10,Oct,4,Q4 2017,2017-12-31,Canada,All fuel types,"Total, vehicle type",Units,2021A000011124,450479.000000
2018-01,2018,1,Jan,1,Q1 2018,2018-03-31,Canada,All fuel types,"Total, vehicle type",Units,2021A000011124,406109.000000


Nunavut coverage snapshot

Identify years with missing `value` in Nunavut, plus years that are fully available for context.

In [24]:
# Nunavut coverage: which years have missing `value`?
nunavut = zev_clean[zev_clean["geo"] == "Nunavut"]

# years where some rows are missing `value`
nunavut_missing_years = (
    nunavut[nunavut["value"].isna()]["Year"]
    .dropna()
    .unique()
    .astype(int)
    .tolist()
)
print("Nunavut years with missing `value`s:", sorted(nunavut_missing_years))

# years that are fully available (no missing)
nunavut_full_years = (
    nunavut.loc[nunavut.groupby("Year")["value"].transform(lambda s: s.notna().all()), "Year"]
    .dropna()
    .unique()
    .astype(int)
    .tolist()
)
print("Nunavut years fully available:", sorted(nunavut_full_years))

Nunavut years with missing `value`s: [2017, 2018, 2019, 2020, 2021, 2022, 2025]
Nunavut years fully available: [2023, 2024]


Filter sparse provinces and tidy categories

Remove provinces with insufficient data, drop helper columns, and keep concrete vehicle/fuel categories to avoid double counting. Tag national vs provincial rows for easier filtering in visuals.

In [25]:
# provinces to remove (insufficient or mostly missing data)
DROP_PROVINCES = ["Alberta", "Newfoundland and Labrador", "Nunavut"]

# filter out selected provinces
zev_processed = zev_processed[~zev_processed["geo"].isin(DROP_PROVINCES)].copy()

# remove helper/metadata we don't want downstream
zev_processed = zev_processed.drop(columns=["value_missing", "value_filled_0"], errors="ignore")
zev_processed = zev_processed.drop(columns=["uom"], errors="ignore")  # typically constant "Units"

# keep specific vehicle/fuel categories (avoid aggregates)
zev_processed = zev_processed[zev_processed["vehicle_type"] != "Total, vehicle type"].copy()
drop_fuels = ["All fuel types", "All zero-emission vehicles"]
zev_processed = zev_processed[~zev_processed["fuel_type"].isin(drop_fuels)].copy()

# tag geography level for easy filtering
zev_processed["is_national"] = zev_processed["geo"].eq("Canada")
zev_processed["geo_level"]   = np.where(zev_processed["is_national"], "national", "province")

# order quarter-representative months for cleaner visuals
month_order = ["Jan", "Apr", "Jul", "Oct"]
if "Month_Name" in zev_processed.columns:
    zev_processed["Month_Name"] = pd.Categorical(
        zev_processed["Month_Name"], categories=month_order, ordered=True
    )

# compact sanity prints
print(f"Removed provinces: {DROP_PROVINCES}")
print("Remaining geographies:", sorted(zev_processed['geo'].dropna().unique().tolist()))
print("Final shape:", zev_processed.shape)
print("Remaining NaN in 'value' (%):", round(zev_processed["value"].isna().mean() * 100, 2))
print("\nRows by geo_level:\n", zev_processed["geo_level"].value_counts())

Removed provinces: ['Alberta', 'Newfoundland and Labrador', 'Nunavut']
Remaining geographies: ['British Columbia', 'Canada', 'Manitoba', 'New Brunswick', 'Northwest Territories', 'Nova Scotia', 'Ontario', 'Prince Edward Island', 'Quebec', 'Saskatchewan', 'Yukon']
Final shape: (8976, 14)
Remaining NaN in 'value' (%): 0.0

Rows by geo_level:
 geo_level
province    8160
national     816
Name: count, dtype: int64


### Build a clean quarterly ZEV series per province  
We narrow the StatCan data to provinces only (no national totals), keep **Battery electric** and **Plug-in hybrid electric**, and use the **Total, vehicle type** row so quarter counts aren’t split by body type. Then we aggregate BEV+PHEV into one ZEV count per province–year–quarter, create a complete grid for 2017–2024, and join the observed values. A wide view helps check whether all four quarters exist; when they do we compute an annual total. For gaps inside a province’s known history, we linearly interpolate quarters and also compute an “imputed” annual sum when at least one quarter is known. Finally, we pick the annual value: prefer a complete-year sum; otherwise use the imputed one.

In [26]:
# focus on provinces (exclude national totals), keep only BEV + PHEV, use the total vehicle-type row,
# and restrict to the analysis window; keep exactly the fields we need and give them tidy names
EV_FUELS = ["Battery electric", "Plug-in hybrid electric"]
YEARS = list(range(2017, 2025))     # 2017..2024
QTRS  = [1, 2, 3, 4]

z = (
    zev_clean.loc[
        (zev_clean["geo"] != "Canada")
        & (zev_clean["fuel_type"].isin(EV_FUELS))
        & (zev_clean["vehicle_type"].eq("Total, vehicle type"))
        & (zev_clean["Year"].between(2017, 2024)),
        ["geo", "Year", "Quarter_Num", "value"]
    ]
    .rename(columns={"Year": "year", "Quarter_Num": "quarter", "value": "zev_count"})
    .copy()
)

# combine BEV + PHEV so each province–year–quarter has a single ZEV count
# (min_count=1 preserves NaN where both components are missing)
z_agg = (
    z.groupby(["geo", "year", "quarter"], as_index=False, dropna=False)["zev_count"]
     .sum(min_count=1)
)

# build a complete province×year×quarter grid, then left-join observed values onto it
geos = z_agg["geo"].dropna().unique()
grid = (
    pd.MultiIndex.from_product([geos, YEARS, QTRS], names=["geo", "year", "quarter"])
      .to_frame(index=False)
)
q = (
    grid.merge(z_agg, on=["geo", "year", "quarter"], how="left")
        .sort_values(["geo", "year", "quarter"])
)

# create a wide view to check quarterly completeness and compute annual sums only when all 4 quarters exist
wide_raw = (
    q.pivot(index=["geo", "year"], columns="quarter", values="zev_count")
     .rename(columns={1: "zev_q1", 2: "zev_q2", 3: "zev_q3", 4: "zev_q4"})
)
quarters_present = wide_raw.notna().sum(axis=1).rename("quarters_present")
annual_complete  = wide_raw.sum(axis=1).where(quarters_present == 4).rename("annual_complete")

# fill internal gaps within each province’s time series only (no forward/backfill beyond known spans)
q_sorted = q.sort_values(["geo", "year", "quarter"]).copy()
q_sorted["zev_count_filled"] = (
    q_sorted.groupby("geo", group_keys=False)["zev_count"]
            .apply(lambda s: s.interpolate(method="linear", limit_area="inside"))
)

# pivot the interpolated values to wide and form an “imputed” annual sum whenever at least one quarter is known
wide_fill = (
    q_sorted.pivot(index=["geo", "year"], columns="quarter", values="zev_count_filled")
            .rename(columns={1: "zev_q1i", 2: "zev_q2i", 3: "zev_q3i", 4: "zev_q4i"})
)
annual_imputed = wide_fill.sum(axis=1, min_count=1).rename("annual_imputed")

# assemble a tidy table and choose the annual measure: prefer complete-year sums, otherwise use the imputed one
one = (
    wide_raw.join([quarters_present, wide_fill, annual_complete, annual_imputed])
            .reset_index()
            .sort_values(["geo", "year"])
)
one["final_annual"] = one["annual_complete"].where(one["annual_complete"].notna(),
                                                   one["annual_imputed"])
one["final_method"] = np.where(
    one["annual_complete"].notna(), "complete_year",
    np.where(one["annual_imputed"].notna(), "imputed_internal", "missing")
)

# keep the analysis window explicit (the grid already enforces this; this is an extra guard)
one = one[one["year"].between(2017, 2024)].copy()


### Quick data promises we want to hold ourselves to  
Alberta should remain missing across 2017–2024 (that’s how the source is). For any rows that *do* have all four quarters, the annual total must equal the sum of those quarters. These asserts guard against accidental changes.


In [27]:
# Alberta must remain missing for 2017–2024
ab = one.query("geo == 'Alberta' and 2017 <= year <= 2024")
assert ab[["zev_q1","zev_q2","zev_q3","zev_q4"]].isna().all(axis=None)
assert (ab["quarters_present"] == 0).all()
assert ab["final_annual"].isna().all()

# Where quarters_present==4, annual_complete should equal sum of the 4 quarters
full = one.query("quarters_present == 4").copy()
calc = full[["zev_q1","zev_q2","zev_q3","zev_q4"]].sum(axis=1)
assert np.allclose(calc.fillna(0), full["annual_complete"].fillna(0))

### Make a tidy ZEV table ready to save  
We keep only the columns that matter, sort by province and year, and coerce a couple of fields to proper integer types. A tiny group-by at the end double-checks that our keys are unique before saving.


In [28]:
# tidy up before saving
cols = [
    "geo", "year",
    "zev_q1", "zev_q2", "zev_q3", "zev_q4",
    "quarters_present",
    "zev_q1i", "zev_q2i", "zev_q3i", "zev_q4i",
    "annual_complete", "annual_imputed",
    "final_annual", "final_method",
]
one_out = (
    one.loc[:, [c for c in cols if c in one.columns]]
       .sort_values(["geo", "year"])
       .assign(
           year=lambda d: d["year"].astype("Int64"),
           quarters_present=lambda d: d["quarters_present"].astype("Int64"),
       )
)

# quick integrity checks
_ = one_out.groupby(["geo", "year"]).size()

### Save the ZEV file  
Write the cleaned ZEV table to disk with a simple preview.


In [29]:
# Save
ZEV_ONE = OUT_DIR / "zev_processed.csv"
one_out.to_csv(ZEV_ONE, index=False)

print("Saved to:", ZEV_ONE)
print("Rows:", len(one_out), "| Columns:", len(one_out.columns))
print(one_out.columns.tolist())
display(one_out.head(10))

Saved to: /content/data/processed/zev_processed.csv
Rows: 104 | Columns: 15
['geo', 'year', 'zev_q1', 'zev_q2', 'zev_q3', 'zev_q4', 'quarters_present', 'zev_q1i', 'zev_q2i', 'zev_q3i', 'zev_q4i', 'annual_complete', 'annual_imputed', 'final_annual', 'final_method']


,geo,year,zev_q1,zev_q2,zev_q3,zev_q4,quarters_present,zev_q1i,zev_q2i,zev_q3i,zev_q4i,annual_complete,annual_imputed,final_annual,final_method
0,Alberta,2017,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
1,Alberta,2018,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
2,Alberta,2019,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
3,Alberta,2020,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
4,Alberta,2021,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
5,Alberta,2022,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
6,Alberta,2023,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
7,Alberta,2024,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
8,British Columbia,2017,785.0,761.0,806.0,790.0,4,785.0,761.0,806.0,790.0,3142.0,3142.0,3142.0,complete_year
9,British Columbia,2018,1367.0,2463.0,2451.0,2041.0,4,1367.0,2463.0,2451.0,2041.0,8322.0,8322.0,8322.0,complete_year


# **Merging chargers × ZEV datasets.**

### Bring chargers and ZEV together  
Load the two processed files, align the join keys (`geo`, `year`), keep the charger fields we actually use downstream, and left-join the ZEV table so every charger row keeps its place. One assert guarantees there’s exactly one row per province–year. For readability, we also rename the chosen annual ZEV metric.


In [30]:
# read the two processed files
zev_one      = pd.read_csv(OUT_DIR / "zev_processed.csv")
chargers_one = pd.read_csv(OUT_DIR / "chargers_processed.csv")

# make keys consistent (types & names)
zev_one["geo"]  = zev_one["geo"].astype(str)
zev_one["year"] = zev_one["year"].astype(int)

chargers_one["geo"]  = chargers_one["geo"].astype(str)
chargers_one["year"] = chargers_one["year"].astype(int)

# keep just the charger fields we actually need in the master
charger_cols = [
    "geo","year",
    "stations_opened","level2_opened","dcfast_opened",
    "chargers_stations","level2_ports","dcfast_ports","chargers_ports",
    "fast_share"
]
chargers_one = chargers_one.loc[:, [c for c in charger_cols if c in chargers_one.columns]].copy()

# left join keeps every geo-year we have on the chargers side (switch to 'inner' if you want strict overlap)
master = (chargers_one
          .merge(zev_one, on=["geo","year"], how="left", suffixes=("", "_zev"))
          .sort_values(["geo","year"])
          .reset_index(drop=True))

# tiny sanity: one row per geo-year
assert master.groupby(["geo","year"]).size().max() == 1

# friendly aliases for modeling/plots
master = master.rename(columns={
    "final_annual": "ev_annual",
    "final_method": "ev_method"
})

display(master.head(12))

,geo,year,stations_opened,level2_opened,dcfast_opened,chargers_stations,level2_ports,dcfast_ports,chargers_ports,fast_share,zev_q1,zev_q2,zev_q3,zev_q4,quarters_present,zev_q1i,zev_q2i,zev_q3i,zev_q4i,annual_complete,annual_imputed,ev_annual,ev_method
0,Alberta,2017,14,29,0,64,123,16,139,0.115108,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
1,Alberta,2018,10,31,0,74,154,16,170,0.094118,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
2,Alberta,2019,30,94,23,104,248,39,287,0.135889,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
3,Alberta,2020,22,29,30,126,277,69,346,0.199422,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
4,Alberta,2021,51,108,37,177,385,106,491,0.215886,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
5,Alberta,2022,153,280,45,330,665,151,816,0.185049,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
6,Alberta,2023,210,434,91,540,1099,242,1341,0.180462,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
7,Alberta,2024,166,323,93,706,1422,335,1757,0.190666,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,missing
8,British Columbia,2017,42,87,19,193,398,49,447,0.109620,785.0,761.0,806.0,790.0,4,785.0,761.0,806.0,790.0,3142.0,3142.0,3142.0,complete_year
9,British Columbia,2018,68,126,47,261,524,96,620,0.154839,1367.0,2463.0,2451.0,2041.0,4,1367.0,2463.0,2451.0,2041.0,8322.0,8322.0,8322.0,complete_year


### Final polish on the merged dataset  
Give the merged table a sensible column order, normalize integer and float dtypes, and run a few safety checks: one row per province–year, years within the analysis window, and charger stock columns never decreasing within a province (they’re cumulative).


In [31]:
# map ZEV final_annual to a better name
if "final_annual" in master.columns and "ev_annual" not in master.columns:
    master["ev_annual"] = pd.to_numeric(master["final_annual"], errors="coerce")
    master["ev_method"] = master.get("final_method")

# preferred column order (keeps only what exists)
col_order = [
    "geo","year",
    "stations_opened","level2_opened","dcfast_opened",
    "chargers_stations","level2_ports","dcfast_ports","chargers_ports","fast_share",
    "zev_q1","zev_q2","zev_q3","zev_q4","quarters_present",
    "zev_q1i","zev_q2i","zev_q3i","zev_q4i",
    "annual_complete","annual_imputed","ev_annual","ev_method",
]
master = master[[c for c in col_order if c in master.columns]].copy()

# light dtype polish
int_cols = ["stations_opened","level2_opened","dcfast_opened",
            "chargers_stations","level2_ports","dcfast_ports","chargers_ports",
            "quarters_present"]
for c in [c for c in int_cols if c in master.columns]:
    master[c] = pd.to_numeric(master[c], errors="coerce").fillna(0).astype(int)

float_cols = ["fast_share","zev_q1","zev_q2","zev_q3","zev_q4",
              "zev_q1i","zev_q2i","zev_q3i","zev_q4i",
              "annual_complete","annual_imputed","ev_annual"]
for c in [c for c in float_cols if c in master.columns]:
    master[c] = pd.to_numeric(master[c], errors="coerce")

# tiny QA: keys unique, years in range, cumulative non-decreasing
assert master.groupby(["geo","year"]).size().max() == 1
assert master["year"].between(2017, 2024).all()
for c in ["chargers_stations","level2_ports","dcfast_ports","chargers_ports"]:
    if c in master.columns:
        ok = master.groupby("geo")[c].diff().fillna(0).ge(0).all()
        assert ok, f"{c} decreases somewhere"

### Save the single, analysis-ready file  
Export the merged dataset with a descriptive name so it’s clear this file contains both charger stocks and ZEV counts for 2017–2024.


In [32]:
# Save
MASTER_PATH = OUT_DIR / "merge_chargers_zev_2017_2024.csv"
master.to_csv(MASTER_PATH, index=False)
print(f"Saved: {MASTER_PATH}  |  rows={len(master)}, cols={master.shape[1]}")

Saved: /content/data/processed/merge_chargers_zev_2017_2024.csv  |  rows=96, cols=23


# Population Dataset

Load & compact schema

Load the raw population file and show a concise schema: shape, columns, dtypes, a small sample, and unique-count summary.

In [33]:
# Load the raw dataset
pop_raw = pd.read_csv(RAW_POP)

# Quick shape
print(f"Dataset shape: {pop_raw.shape[0]:,} rows × {pop_raw.shape[1]} columns\n")

# Column names
print("Columns:")
for col in pop_raw.columns:
    print(" -", col)

# Data types
print("\nData types:")
print(pop_raw.dtypes)

# Small sample
print("\nSample data:")
display(pop_raw.head())

# Unique counts per column (structure awareness)
print("\nUnique counts per column:")
for col in pop_raw.columns:
    print(f"{col}: {pop_raw[col].nunique(dropna=False)} unique values")

Dataset shape: 1,134 rows × 16 columns

Columns:
 - REF_DATE
 - GEO
 - DGUID
 - Gender
 - Age group
 - UOM
 - UOM_ID
 - SCALAR_FACTOR
 - SCALAR_ID
 - VECTOR
 - COORDINATE
 - VALUE
 - STATUS
 - SYMBOL
 - TERMINATED
 - DECIMALS

Data types:
REF_DATE           int64
GEO               object
DGUID             object
Gender            object
Age group         object
UOM               object
UOM_ID             int64
SCALAR_FACTOR     object
SCALAR_ID          int64
VECTOR            object
COORDINATE        object
VALUE              int64
STATUS           float64
SYMBOL           float64
TERMINATED       float64
DECIMALS           int64
dtype: object

Sample data:


,REF_DATE,GEO,DGUID,Gender,Age group,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2017,Canada,2021A000011124,Total - gender,All ages,Persons,249,units,0,v466668,1.1.1,36545075,NaN,NaN,NaN,0
1,2018,Canada,2021A000011124,Total - gender,All ages,Persons,249,units,0,v466668,1.1.1,37072620,NaN,NaN,NaN,0
2,2019,Canada,2021A000011124,Total - gender,All ages,Persons,249,units,0,v466668,1.1.1,37618495,NaN,NaN,NaN,0
3,2020,Canada,2021A000011124,Total - gender,All ages,Persons,249,units,0,v466668,1.1.1,38028638,NaN,NaN,NaN,0
4,2021,Canada,2021A000011124,Total - gender,All ages,Persons,249,units,0,v466668,1.1.1,38239864,NaN,NaN,NaN,0



Unique counts per column:
REF_DATE: 9 unique values
GEO: 14 unique values
DGUID: 14 unique values
Gender: 3 unique values
Age group: 3 unique values
UOM: 1 unique values
UOM_ID: 1 unique values
SCALAR_FACTOR: 1 unique values
SCALAR_ID: 1 unique values
VECTOR: 126 unique values
COORDINATE: 126 unique values
VALUE: 1131 unique values
STATUS: 1 unique values
SYMBOL: 1 unique values
TERMINATED: 1 unique values
DECIMALS: 1 unique values


Select columns for population analysis

Keep only the fields we use downstream and rename them for consistency.

In [34]:
# Select columns for population analysis
# Keep these
keep_cols = ["REF_DATE", "GEO", "DGUID", "Gender", "Age group", "VALUE"]

# Drop all others
pop_clean = pop_raw[keep_cols].copy()

# Rename to consistent, lowercase names
pop_clean.columns = ["year", "geo", "dguid", "gender", "age_group", "value"]

# Preview result
print(f"Cleaned dataset shape: {pop_clean.shape[0]:,} rows × {pop_clean.shape[1]} columns\n")
display(pop_clean.head(10))

# Dtypes and missingness
print("\nData types:")
print(pop_clean.dtypes)

print("\nMissing values per column:")
print(pop_clean.isna().sum())


Cleaned dataset shape: 1,134 rows × 6 columns



,year,geo,dguid,gender,age_group,value
0,2017,Canada,2021A000011124,Total - gender,All ages,36545075
1,2018,Canada,2021A000011124,Total - gender,All ages,37072620
2,2019,Canada,2021A000011124,Total - gender,All ages,37618495
3,2020,Canada,2021A000011124,Total - gender,All ages,38028638
4,2021,Canada,2021A000011124,Total - gender,All ages,38239864
5,2022,Canada,2021A000011124,Total - gender,All ages,38950132
6,2023,Canada,2021A000011124,Total - gender,All ages,40049088
7,2024,Canada,2021A000011124,Total - gender,All ages,41262329
8,2025,Canada,2021A000011124,Total - gender,All ages,41651653
9,2017,Canada,2021A000011124,Total - gender,16 to 64 years,24103589



Data types:
year          int64
geo          object
dguid        object
gender       object
age_group    object
value         int64
dtype: object

Missing values per column:
year         0
geo          0
dguid        0
gender       0
age_group    0
value        0
dtype: int64


Inspect key categoricals

Review geography, age groups, and gender values with compact counts for a quick sense of structure.

In [35]:
# Inspect key categorical columns

print("Unique GEO values:\n", pop_clean["geo"].unique(), "\n")
print("Unique AGE_GROUP values:\n", pop_clean["age_group"].unique(), "\n")
print("Unique GENDER values:\n", pop_clean["gender"].unique(), "\n")

# Check counts per category for clarity
print("Counts per GEO:")
print(pop_clean["geo"].value_counts())

print("\nCounts per AGE_GROUP:")
print(pop_clean["age_group"].value_counts())

print("\nCounts per GENDER:")
print(pop_clean["gender"].value_counts())


Unique GEO values:
 ['Canada' 'Newfoundland and Labrador' 'Prince Edward Island' 'Nova Scotia'
 'New Brunswick' 'Quebec' 'Ontario' 'Manitoba' 'Saskatchewan' 'Alberta'
 'British Columbia' 'Yukon' 'Northwest Territories' 'Nunavut'] 

Unique AGE_GROUP values:
 ['All ages' '16 to 64 years' '65 years and older'] 

Unique GENDER values:
 ['Total - gender' 'Men+' 'Women+'] 

Counts per GEO:
geo
Canada                       81
Newfoundland and Labrador    81
Prince Edward Island         81
Nova Scotia                  81
New Brunswick                81
Quebec                       81
Ontario                      81
Manitoba                     81
Saskatchewan                 81
Alberta                      81
British Columbia             81
Yukon                        81
Northwest Territories        81
Nunavut                      81
Name: count, dtype: int64

Counts per AGE_GROUP:
age_group
All ages              378
16 to 64 years        378
65 years and older    378
Name: count, dtype: int6

Filter to driving-age population (16+)

Keep 16–64 and 65+ groups, preserve gender splits, and rename the metric to `population`.

In [36]:
# Filter to driving-age population (16+)

pop_filtered = pop_clean[
    pop_clean["age_group"].isin(["16 to 64 years", "65 years and older"])
].copy()

# Rename value column to 'population' for clarity
pop_filtered.rename(columns={"value": "population"}, inplace=True)

# Preview result
print(f"Final population dataset: {pop_filtered.shape[0]:,} rows × {pop_filtered.shape[1]} columns\n")
display(pop_filtered.head())

# Verify unique values
print("\nUnique AGE_GROUPS:", pop_filtered["age_group"].unique())
print("Unique GENDERS:", pop_filtered["gender"].unique())


Final population dataset: 756 rows × 6 columns



,year,geo,dguid,gender,age_group,population
9,2017,Canada,2021A000011124,Total - gender,16 to 64 years,24103589
10,2018,Canada,2021A000011124,Total - gender,16 to 64 years,24345842
11,2019,Canada,2021A000011124,Total - gender,16 to 64 years,24590541
12,2020,Canada,2021A000011124,Total - gender,16 to 64 years,24719666
13,2021,Canada,2021A000011124,Total - gender,16 to 64 years,24696494



Unique AGE_GROUPS: ['16 to 64 years' '65 years and older']
Unique GENDERS: ['Total - gender' 'Men+' 'Women+']


Sanity checks on the filtered population

Confirm no obvious data issues: missingness, duplicates at the kept granularity, and coverage by geo × year × gender.

In [37]:
# Sanity checks on filtered population (16+)
# Basic integrity
print("Rows:", len(pop_filtered))
print("Missing % by col:")
display(pop_filtered.isna().mean().round(3) * 100)

# Duplicates at the most granular level we keep
dup_keys = ["year", "geo", "dguid", "gender", "age_group"]
dup_count = pop_filtered.duplicated(subset=dup_keys).sum()
print("\nExact-duplicate rows on", dup_keys, "-", dup_count)

# Quick coverage by geo × year × gender
cov = (
    pop_filtered.groupby(["geo","year","gender"], dropna=False)
                .size().rename("rows").reset_index()
                .sort_values(["geo","year","gender"])
)
print("\nCoverage preview (geo × year × gender):")
display(cov.head())

Rows: 756
Missing % by col:


,0
year,0.0
geo,0.0
dguid,0.0
gender,0.0
age_group,0.0
population,0.0



Exact-duplicate rows on ['year', 'geo', 'dguid', 'gender', 'age_group'] - 0

Coverage preview (geo × year × gender):


,geo,year,gender,rows
0,Alberta,2017,Men+,2
1,Alberta,2017,Total - gender,2
2,Alberta,2017,Women+,2
3,Alberta,2018,Men+,2
4,Alberta,2018,Total - gender,2


Build a tidy 16+ population table

Aggregate the two age bands into a single `16_plus` bucket (keeping gender), and add simple geography flags for Tableau filters.

In [38]:
# Build a tidy 16+ population table
# Combine the two age groups into the '16_plus' bucket (keep gender)
pop_16plus = (
    pop_filtered.assign(age="16_plus")
                .groupby(["year","geo","dguid","gender","age"], as_index=False, dropna=False)
                .agg(population=("population","sum"))
)

# Add simple geo flags for filtering (national vs province)
pop_16plus["geo_level"]   = np.where(pop_16plus["geo"]=="Canada", "national", "province")
pop_16plus["is_national"] = pop_16plus["geo"].eq("Canada")

# Ensure numeric types
pop_16plus["year"]       = pop_16plus["year"].astype("Int64")
pop_16plus["population"] = pop_16plus["population"].astype("Int64")

print("pop_16plus shape:", pop_16plus.shape)
display(pop_16plus.head())

print("\nChecks:")
print("- genders:", pop_16plus["gender"].unique().tolist())
print("- geo levels:", pop_16plus["geo_level"].value_counts().to_dict())

pop_16plus shape: (378, 8)


,year,geo,dguid,gender,age,population,geo_level,is_national
0,2017,Alberta,2021A000248,Men+,16_plus,1701045,province,False
1,2017,Alberta,2021A000248,Total - gender,16_plus,3387024,province,False
2,2017,Alberta,2021A000248,Women+,16_plus,1685979,province,False
3,2017,British Columbia,2021A000259,Men+,16_plus,2052438,province,False
4,2017,British Columbia,2021A000259,Total - gender,16_plus,4173138,province,False



Checks:
- genders: ['Men+', 'Total - gender', 'Women+']
- geo levels: {'province': 351, 'national': 27}


Keep age buckets and shares

This keeps the two age groups (16–64, 65+) and computes their shares per province–year.

In [39]:
# Build a by-age table (keep gender here to respect the source)
pop_by_age = (
    pop_filtered
    .groupby(["year", "geo", "dguid", "gender", "age_group"], dropna=False, as_index=False)
    .agg(population=("population", "sum"))
)

# Aggregate across gender to get total per age group
age_totals = (
    pop_by_age.groupby(["year", "geo", "age_group"], as_index=False)["population"]
              .sum()
)

# Pivot to wide: one column per age group
age_pct = (
    age_totals.pivot(index=["year", "geo"], columns="age_group", values="population")
              .reset_index()
)

# Compute percentages (0–100). Works even if one bucket is missing.
denom = (
    age_pct.get("16 to 64 years", 0).fillna(0) +
    age_pct.get("65 years and older", 0).fillna(0)
)
with np.errstate(divide="ignore", invalid="ignore"):
    pct_65 = (age_pct.get("65 years and older", 0).fillna(0) / denom) * 100
    pct_16 = 100 - pct_65

age_pct["pct_16_64"]   = pct_16.round(2)
age_pct["pct_65_plus"] = pct_65.round(2)

# Order columns neatly (keep counts + percents)
ordered_cols = ["year", "geo"]
for col in ["16 to 64 years", "65 years and older"]:
    if col in age_pct.columns:
        ordered_cols.append(col)
ordered_cols += ["pct_16_64", "pct_65_plus"]
age_pct = age_pct[[c for c in ordered_cols if c in age_pct.columns]]

# Save as a sidecar file (numeric percentages; no '%' symbol in the data)
age_pct_out = OUT_DIR / "population_age_shares.csv"
age_pct.to_csv(age_pct_out, index=False)
print(f"Saved age percentages to: {age_pct_out}")

# Compact preview with % formatting
display(
    age_pct.head(10)
           .style.hide(axis="index")
           .format({"pct_16_64": "{:.2f}%", "pct_65_plus": "{:.2f}%"})
           .set_caption("Age composition by province–year (percent)")
)

Saved age percentages to: /content/data/processed/population_age_shares.csv


year,geo,16 to 64 years,65 years and older,pct_16_64,pct_65_plus
2017,Alberta,5725842,1048206,84.53%,15.47%
2017,British Columbia,6591460,1754816,78.97%,21.03%
2017,Canada,48207178,12250966,79.74%,20.26%
2017,Manitoba,1728112,404212,81.04%,18.96%
2017,New Brunswick,987206,308774,76.17%,23.83%
2017,Newfoundland and Labrador,691818,208982,76.80%,23.20%
2017,Northwest Territories,63374,6762,90.36%,9.64%
2017,Nova Scotia,1236392,378684,76.55%,23.45%
2017,Nunavut,47136,2858,94.28%,5.72%
2017,Ontario,18657796,4676040,79.96%,20.04%


Save processed population table

Write a tidy population file with year, geography, gender, and the `16_plus` bucket, ready for joining or Tableau.

In [40]:
# Save processed population dataset
# Desired column order for downstream joins
cols_order = [
    "year", "geo", "dguid", "geo_level", "is_national",
    "gender", "age", "population"
]
pop_final = pop_16plus[cols_order].copy()

# Write to disk
out_path = OUT_DIR / "population_processed.csv"
pop_final.to_csv(out_path, index=False)

print(f"Saved population to: {out_path}")
print(f"Rows: {len(pop_final):,} | Columns: {len(pop_final.columns)}")

## Small preview
display(pop_final.head())


Saved population to: /content/data/processed/population_processed.csv
Rows: 378 | Columns: 8


,year,geo,dguid,geo_level,is_national,gender,age,population
0,2017,Alberta,2021A000248,province,False,Men+,16_plus,1701045
1,2017,Alberta,2021A000248,province,False,Total - gender,16_plus,3387024
2,2017,Alberta,2021A000248,province,False,Women+,16_plus,1685979
3,2017,British Columbia,2021A000259,province,False,Men+,16_plus,2052438
4,2017,British Columbia,2021A000259,province,False,Total - gender,16_plus,4173138


# MERGE

### Load processed datasets and show structure
Read the processed ZEV, chargers, and population files. Print shapes and columns for a quick schema check.

In [41]:
# Paths to processed files
PROC_ZEV = OUT_DIR / "zev_processed_quarterly.csv"
PROC_CHG = OUT_DIR / "chargers_processed.csv"
PROC_POP = OUT_DIR / "population_processed.csv"

zev = pd.read_csv(PROC_ZEV)
chg = pd.read_csv(PROC_CHG)
pop = pd.read_csv(PROC_POP)

# Shapes
print("Shapes:")
print("zev:", zev.shape)
print("chg:", chg.shape)
print("pop:", pop.shape)

# Column names (quick scan)
print("\nColumn names:")
print("ZEV cols:", zev.columns.tolist())
print("CHG cols:", chg.columns.tolist())
print("POP cols:", pop.columns.tolist())


FileNotFoundError: [Errno 2] No such file or directory: '/content/data/processed/zev_processed_quarterly.csv'

### Population — provincial 16+ totals (denominator)
Extract provincial rows for the `16_plus` age bucket with the total-gender line to use as denominators.

In [ ]:
# Required columns check (population file)
req_cols = ["year","geo","geo_level","is_national","gender","age","population"]
assert all(c in pop.columns for c in req_cols), "population_processed.csv is missing expected columns."

# Keep provincial, total-gender, 16+ rows only
pop_prov = (
    pop.query("is_national == False & gender == 'Total - gender' & age == '16_plus'")
       .loc[:, ["year","geo","population"]]
       .rename(columns={"population":"population_16plus"})
       .copy()
)

# Quick sanity readout
print("pop_prov:", pop_prov.shape)
print("years:", pop_prov["year"].min(), "-", pop_prov["year"].max())
print("geos:", sorted(pop_prov["geo"].unique()))
display(pop_prov.head())

### Zero Emission Vehicles (ZEV) — annual EV totals and BEV/PHEV split
Use quarterly ZEV to build annual counts by province. Keep BEV and PHEV splits and verify they sum to the total.

In [ ]:
# Required columns check (ZEV file)
need = ["Year", "geo", "fuel_type", "value"]
assert all(c in zev.columns for c in need), "zev_processed_quarterly.csv is missing expected columns."

# Keep provinces (drop national rows if present)
zev_prov = zev.copy()
if "is_national" in zev_prov.columns:
    zev_prov = zev_prov[zev_prov["is_national"] == False].copy()

# Drop vehicle-type aggregate to avoid double-counting
if "vehicle_type" in zev_prov.columns:
    zev_prov = zev_prov[zev_prov["vehicle_type"] != "Total, vehicle type"].copy()

# EV fuels (Keep BEV + PHEV only)
ev_fuels = ["Battery electric", "Plug-in hybrid electric"]
zev_ev = zev_prov[zev_prov["fuel_type"].isin(ev_fuels)].copy()

# Sum across quarters and vehicle types to get annual EV totals
ev_annual = (
    zev_ev.groupby(["geo", "Year"], as_index=False)["value"]
          .sum()
          .rename(columns={"Year": "year", "value": "ev_count"})
)

# Compute BEV and PHEV annuals separately
bev = (
    zev_prov[zev_prov["fuel_type"].eq("Battery electric")]
    .groupby(["geo", "Year"], as_index=False)["value"].sum()
    .rename(columns={"Year": "year", "value": "bev_count"})
)
phev = (
    zev_prov[zev_prov["fuel_type"].eq("Plug-in hybrid electric")]
    .groupby(["geo", "Year"], as_index=False)["value"].sum()
    .rename(columns={"Year": "year", "value": "phev_count"})
)

ev_annual = (
    ev_annual.merge(bev,  on=["geo", "year"], how="left")
             .merge(phev, on=["geo", "year"], how="left")
             .fillna({"bev_count": 0, "phev_count": 0})
)

# Ensure integer types and sort
ev_annual["year"] = ev_annual["year"].astype("Int64")
for c in ["ev_count", "bev_count", "phev_count"]:
    ev_annual[c] = ev_annual[c].astype("Int64")

ev_annual = ev_annual.sort_values(["geo", "year"]).reset_index(drop=True)

# Sanity check: EV total equals BEV + PHEV
mismatch = (ev_annual["ev_count"] != (ev_annual["bev_count"] + ev_annual["phev_count"])).sum()
assert mismatch == 0, f"Found {mismatch} rows where ev_count != bev+phev"

print("ev_annual:", ev_annual.shape)
print("years:", ev_annual["year"].min(), "-", ev_annual["year"].max())
print("geos:", sorted(ev_annual["geo"].unique()))
display(ev_annual.head())


### Chargers — annual provincial aggregates
Filter to electric, public, active stations. Aggregate annual station counts and connector counts by province.

In [ ]:
# Chargers — annual provincial aggregates
chg_need = ["State", "Open Year", "Fuel Type Code", "Access Code", "Status Code",
            "EV Level2 EVSE Num", "EV DC Fast Count"]
assert all(c in chg.columns for c in chg_need), "chargers_processed.csv missing expected columns."

# Filter: electric, public access, active status
chg_filt = (
    chg.loc[
        (chg["Fuel Type Code"].str.upper() == "ELEC") &
        (chg["Access Code"].str.lower() == "public") &
        (chg["Status Code"].str.upper() == "E")
    , ["State","Open Year","EV Level2 EVSE Num","EV DC Fast Count"]]
    .copy()
)

# Rename to (geo, year) and coerce counts to numeric
chg_filt.rename(columns={"State":"geo", "Open Year":"year"}, inplace=True)
for c in ["EV Level2 EVSE Num", "EV DC Fast Count"]:
    chg_filt[c] = pd.to_numeric(chg_filt[c], errors="coerce").fillna(0)

chg_filt["year"] = pd.to_numeric(chg_filt["year"], errors="coerce").astype("Int64")
chg_filt = chg_filt.dropna(subset=["year"])  # drop rows without a valid year

# Aggregate annual station and connector counts
chargers_annual = (
    chg_filt
      .groupby(["geo","year"], as_index=False)
      .agg(
          stations_count = ("geo", "size"),                   # number of public, operating sites opened that year
          level2_ports   = ("EV Level2 EVSE Num","sum"),      # sum of Level-2 connectors
          dcfc_ports     = ("EV DC Fast Count","sum")         # sum of DC fast connectors
      )
      .sort_values(["geo","year"])
      .reset_index(drop=True)
)

# Cast to integers for tidy output
for c in ["stations_count","level2_ports","dcfc_ports"]:
    chargers_annual[c] = chargers_annual[c].astype("Int64")

print("chargers_annual:", chargers_annual.shape)
print("years:", chargers_annual["year"].min(), "-", chargers_annual["year"].max())
print("geos:", sorted(chargers_annual["geo"].unique()))
display(chargers_annual.head())

### Chargers — openings to in-service inventory (cumulative)
Fill the province×year grid so no years are skipped, treat missing openings as 0, and build a cumulative **stations_in_service** series per province.

In [ ]:
# Chargers — convert annual openings to cumulative in-service inventory

# Complete (geo × year) grid so no years are skipped
yr_min, yr_max = int(chargers_annual["year"].min()), int(chargers_annual["year"].max())
geos = sorted(chargers_annual["geo"].unique())
full_idx = pd.MultiIndex.from_product([geos, range(yr_min, yr_max + 1)],
                                      names=["geo", "year"])

inventory = (
    chargers_annual
      .set_index(["geo", "year"])
      .reindex(full_idx)      # insert missing (geo, year) with NaN
      .fillna(0)              # treat missing as 0 openings
      .reset_index()
)

# Cumulative stations in service within each province
inventory["stations_count"] = (
    inventory.groupby("geo")["stations_count"].cumsum().astype("Int64")
)

# Keep tidy stations-only column
inventory = inventory.rename(columns={"stations_count": "stations_in_service"})
chargers_inventory = (
    inventory.loc[:, ["geo", "year", "stations_in_service"]]
             .sort_values(["geo", "year"])
             .reset_index(drop=True)
)

# Align to ZEV window (2017+) for clean merging
chargers_inventory = chargers_inventory[chargers_inventory["year"] >= 2017].copy()

# Sanity - cumulative must be non-decreasing
decreasing_steps = (
    chargers_inventory.groupby("geo")["stations_in_service"]
                      .apply(lambda s: (s.diff().fillna(0) < 0).sum())
                      .sum()
)
assert decreasing_steps == 0, f"Found {decreasing_steps} decreasing steps in stations_in_service!"

# Summary
print("chargers_inventory (stations only):", chargers_inventory.shape)
print("years:", int(chargers_inventory['year'].min()), "-", int(chargers_inventory['year'].max()))
print("geos:", sorted(chargers_inventory["geo"].unique()))
display(chargers_inventory.head(10))


### Chargers — normalize province names
Map any two-letter province codes to full province names to match ZEV and population files.

In [ ]:
# Normalize province names
prov_map = {
    "AB": "Alberta",
    "BC": "British Columbia",
    "MB": "Manitoba",
    "NB": "New Brunswick",
    "NL": "Newfoundland and Labrador",
    "NS": "Nova Scotia",
    "NT": "Northwest Territories",
    "NU": "Nunavut",
    "ON": "Ontario",
    "PE": "Prince Edward Island",
    "QC": "Quebec",
    "SK": "Saskatchewan",
    "YT": "Yukon",
}

chargers_inventory["geo"] = chargers_inventory["geo"].replace(prov_map)

# Quick check
print("Unique geos (chargers):", sorted(chargers_inventory["geo"].unique()))
display(chargers_inventory.head(8))


### Chargers — removal of sparse provinces



In [ ]:
DROP_PROVINCES = ["Alberta", "Newfoundland and Labrador", "Nunavut"]
chargers_inventory_merge = (
    chargers_inventory[~chargers_inventory["geo"].isin(DROP_PROVINCES)]
    .reset_index(drop=True)
)
print("chargers_inventory_merge:", chargers_inventory_merge.shape)
display(chargers_inventory_merge.head())

### Merge EV, population, and chargers (province, annual)
Keep only overlapping years and inner-join on `geo` + `year` so all rows have full coverage across the three sources.


In [ ]:
# Merge EV + Population + Chargers (provinces, annual)

# Required columns
need_ev  = {"geo","year","ev_count","bev_count","phev_count"}
need_pop = {"geo","year","population_16plus"}
need_chg = {"geo","year","stations_in_service"}

assert need_ev.issubset(ev_annual.columns), "ev_annual is missing columns"
assert need_pop.issubset(pop_prov.columns), "pop_prov is missing columns"
assert need_chg.issubset(chargers_inventory_merge.columns), "chargers_inventory_merge is missing columns"

# Common years across all three datasets
yrs_ev  = set(ev_annual["year"].unique())
yrs_pop = set(pop_prov["year"].unique())
yrs_chg = set(chargers_inventory_merge["year"].unique())
years_common = sorted(yrs_ev & yrs_pop & yrs_chg)

ev_use  = ev_annual.query("year in @years_common").copy()
pop_use = pop_prov.query("year in @years_common").copy()
chg_use = chargers_inventory_merge.query("year in @years_common").copy()

# Inner-join on (geo, year) to keep fully covered rows
merged = (
    ev_use.merge(pop_use, on=["geo","year"], how="inner")
          .merge(chg_use,  on=["geo","year"], how="inner")
          .copy()
)

# Ensure integer dtypes and sort
int_cols = ["year","ev_count","bev_count","phev_count","population_16plus","stations_in_service"]
for c in int_cols:
    merged[c] = merged[c].astype("Int64")

merged = merged.sort_values(["geo","year"]).reset_index(drop=True)

# Quick QA
print("merged shape:", merged.shape)
print("years:", int(merged["year"].min()), "-", int(merged["year"].max()))
print("geos:", sorted(merged["geo"].unique()))
print("\nMissing % by column:")
print((merged.isna().mean()*100).round(2))

# Preview
cols_show = ["geo","year","ev_count","bev_count","phev_count","population_16plus","stations_in_service"]
display(merged.head(10)[cols_show])

### Derived metrics for analysis and Tableau
Compute per-capita measures, EVs per station, and BEV/PHEV shares. Keep precise (4-dp) ratios for clean visuals.


In [ ]:
# Derived metrics for Tableau

df = merged.copy()

# Per-capita and ratio metrics
df["ev_per_1000_people"]        = (df["ev_count"] / df["population_16plus"]) * 1000
df["stations_per_100k_people"]  = (df["stations_in_service"] / df["population_16plus"]) * 100000
df["evs_per_station"]           = df["ev_count"] / df["stations_in_service"]

# Market shares (BEV vs PHEV)
df["bev_share"]  = df["bev_count"] / df["ev_count"]
df["phev_share"] = df["phev_count"] / df["ev_count"]

# Round ratios for clean display
ratio_cols = ["ev_per_1000_people","stations_per_100k_people","evs_per_station","bev_share","phev_share"]
df[ratio_cols] = df[ratio_cols].round(4)

# Column order for Tableau
cols_order = [
    "geo","year",
    "population_16plus",
    "ev_count","bev_count","phev_count",
    "stations_in_service",
    "ev_per_1000_people","stations_per_100k_people","evs_per_station",
    "bev_share","phev_share"
]
df = df[cols_order].sort_values(["geo","year"]).reset_index(drop=True)

# Quick QA
print("Final Tableau-ready dataset:", df.shape)
print("years:", int(df["year"].min()), "-", int(df["year"].max()))
print("geos:", sorted(df["geo"].unique()))
display(df.head(10))

### Export Tableau-ready dataset
Write the combined province-year table with counts and derived metrics. We keep current column names; any renames can be applied in Tableau if needed.


In [ ]:
# Export Tableau-ready dataset

OUT_PATH = OUT_DIR / "combined_ev_population_chargers.csv"
df.to_csv(OUT_PATH, index=False)

# Write to disk
print(f"Saved Tableau dataset to: {OUT_PATH}")
print(f"Rows: {len(df):,} | Columns: {len(df.columns)}")

## Quick insights (first pass)
- **Growth:** EV counts and in-service charging stations both rise across provinces from 2017 onward, with acceleration after 2020.
- **Leaders (per capita):** Provinces like **Quebec** and **British Columbia** typically rank high on both `ev_per_1000_people` and `stations_per_100k_people`.
- **BEV vs PHEV:** BEV share (`bev_share`) increases over time in most provinces, suggesting improving charging confidence and model availability.
- **Gaps:** Provinces with low `stations_per_100k_people` but rising `ev_per_1000_people` may face pressure on infrastructure (watch `evs_per_station` for strain).
